# Deriving the determining equations of the Lotka-Volterra model
*Date*: 2022-04-13,<br>
*Written by:* Johannes Borgqvist.<br>
Ok, so consider the dimensionless *Lotka-Volterra* model:
\begin{align}
    \dfrac{\mathrm{d}u}{\mathrm{d}t}&=u(1-v),\\
    \dfrac{\mathrm{d}v}{\mathrm{d}t}&=\alpha v(u-1).\\
\end{align}
Now, we would like to see if we can derive the determining equations from this using sympy. <br>

In [1]:
# Import sympy
from sympy import *
# Translate a string to symbolic expression
from sympy.parsing.sympy_parser import parse_expr

# Setting up the reaction terms and so on

In [2]:
# Independent variable
t = symbols('t')
# Dependent variables
u,v = symbols('u v',cls=Function)
# Define our parameter a
a = symbols('a')
# Define the states
states = [u(t), v(t)]
# Define our reaction terms
omega_list = [u(t)*(1-v(t)), a*v(t)*(u(t)-1)]

print(set(range(1,len(states)+1)))

{1, 2}


# Try to implement linearised symmetry condition brute force style

In [3]:
# Allocate tangents
xi,eta_1,eta_2 = symbols('xi eta_1 eta_2',cls=Function)
# Help variable
x = symbols('x')
# Define the tangents
xi_f = xi(t)
eta_1_f = eta_1(t,u(t),v(t))
eta_2_f = eta_2(t,u(t),v(t))
# Linearised symmetry condition 1
LHS = Derivative(eta_1_f,t).doit() - omega_list[0]*Derivative(xi_f,t).doit()
print("\t\t\tBefore:\n")
print(LHS)
# Replace time derivatives with reaction terms
LHS = LHS.subs(Derivative(u(t), t),omega_list[0])
LHS = LHS.subs(Derivative(v(t), t),omega_list[1])
print("\t\t\tAfter:\n")
print(LHS)
# The RHS
RHS = 0 
for index in range(1,len(states)+1):
    exec("RHS += eta_%d_f*Derivative(omega_list[0].subs(states[index-1],x),x).subs(x,states[index-1]).doit()"%(index))
print("\t\t\tRHS:\n")
print(RHS)
print(latex(LHS-RHS))


			Before:

-(1 - v(t))*u(t)*Derivative(xi(t), t) + Derivative(eta_1(t, u(t), v(t)), u(t))*Derivative(u(t), t) + Derivative(eta_1(t, u(t), v(t)), v(t))*Derivative(v(t), t) + Subs(Derivative(eta_1(_xi_1, u(t), v(t)), _xi_1), _xi_1, t)
			After:

a*(u(t) - 1)*v(t)*Derivative(eta_1(t, u(t), v(t)), v(t)) + (1 - v(t))*u(t)*Derivative(eta_1(t, u(t), v(t)), u(t)) - (1 - v(t))*u(t)*Derivative(xi(t), t) + Subs(Derivative(eta_1(_xi_1, u(t), v(t)), _xi_1), _xi_1, t)
			RHS:

(1 - v(t))*eta_1(t, u(t), v(t)) - eta_2(t, u(t), v(t))*u(t)
a \left(u{\left(t \right)} - 1\right) v{\left(t \right)} \frac{d}{d v{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)} - \left(1 - v{\left(t \right)}\right) \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)} + \left(1 - v{\left(t \right)}\right) u{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)} - \left(1 - v{\left(t \right)}\right) u{\left(t \right)} \frac{d}

# Function for setting up the linearised symmetry condition
So now we will try to set up a general function for deriving the linearised symmetry conditions!

In [4]:
# Function "lin_sym_conds_fibre_preserving". This functions sets up the linearised symmetry conditions
# given an independent variable t, the states and the reaction terms
def lin_sym_conds_fibre_preserving(t,states,reaction_terms):
    # Temporary variables for the dependent variables so that we can 
    # differentiate the reaction terms with respect to each state
    x = symbols('x')    
    # Add a temporary function which we differentiate all the time
    f = symbols('f',cls=Function)
    # Define a list of tangents
    tangent_list = []
    # Define the time tangent
    xi = symbols('xi', cls=Function)
    # Add it to the tangent list
    tangent_list.append(xi(t))
    # Loop over the states and add a tangent for each state, hey?
    for state_index in range(1,len(states)+1):
        # We start by allocating our new tangent as a symbolic function
        #exec("eta_%d = symbols(\'eta_%d\',cls=Function) "%(state_index,state_index)) 
        # Here, we essentially define a string which we then convert
        # to a symbolic expression using "parse_expr"
        temp_str = "eta_" + str(state_index) + "(" + str(t) + "," 
        for state in states:
            temp_str += str(state) + ","
        temp_str += ")"
        temp_str = temp_str.replace(",)",")")
        # Assign this expression to our function f
        #exec("eta_%d_f=%s"%(state_index,temp_str))
        # Add our new tangent
        #exec("tangent_list.append(eta_%d_f)"%(state_index))
        tangent_list.append(parse_expr(temp_str))
    # Ok, so at this point we have our tangent lists, now we can start assembling our linearised symmetry conditions
    # Allocate memory for our linearised symmetry conditions
    lin_sym_original = []
    # Loop over the number of states and define the linearised symmetry condition
    for state_index in range(1,len(states)+1):
        # Define the LHS
        LHS = Derivative(tangent_list[state_index],t).doit() - omega_list[state_index-1]*Derivative(tangent_list[0],t).doit()
        # Allocate memory for the RHS
        RHS = 0
        # Add all partial derivatives of the current reaction term by looping over all states and times
        for RHS_index in range(1,len(states)+1):
            # Extract temporary tangent
            tangent_temp = tangent_list[RHS_index]
            # Partial derivative wrt to the time
            RHS += (Derivative(omega_list[state_index-1].subs(states[RHS_index-1],x),x).subs(x,states[RHS_index-1]).doit()*tangent_temp)
        # After this is done we loop over the states one more time and substitute all time derivatives of the
        # states with the reaction terms
        eq_temp = LHS - RHS
        for state_index in range(len(omega_list)):
            # Also, we substitute the reaction terms in the LHS
            eq_temp = eq_temp.subs(Derivative(states[state_index],t),omega_list[state_index])
        # Append the equation defined as LHS-RHS
        lin_sym_original.append(eq_temp)
    # Return the list with all the tangents
    return tangent_list, lin_sym_original
# Function, latex_printing_of_lin_sym
# This function takes in the time t, the states, tangent_list and the linearised symmetry condition
# and prints them in a nice fashion.
def latex_printing_of_lin_sym(t,states,tangent_list,lin_sym_original):
    # We essentially return a giant string which can be printed, so let's start constructing it
    tangent_str = "\n\nThe tangents are:\n$$" + latex(tangent_list) + "$$\n"
    output_str = ""
    # Get the arguments as well
    argument_str = latex(tangent_list[1]).replace("\\eta_{1}","")
    # Loop through the linearised symmetry condition and add the equations
    output_str += "\nThe linearised symmetry conditions are:\n\n\\begin{align*}\n"
    for lin_sym in lin_sym_original:
        output_str += latex(lin_sym) + "&=0,\\\\\n"
    output_str += "\\end{align*}\n\n"
    # Now we replace the arguments as well to see if this helps
    output_str = output_str.replace(argument_str,"")  
    # Remove time dependence already
    output_str = output_str.replace("{\\left(t \\right)}","")    
    # Loop through the tangents and replace strings for partial derivatives
    for tangent_index in range(len(tangent_list)):
        if tangent_index == 0:
            # Time derivative
            time_str = "\\left. \\frac{\\partial}{\\partial \\xi_{1}} \\xi{\\left(\\xi_{1},u,v \\right)} \\right|_{\\substack{ \\xi_{1}=t }}"
            output_str = output_str.replace(time_str,"\\frac{d\\xi}{dt}")
            output_str = output_str.replace("\\frac{d}{d t} \\xi","\\frac{d\\xi}{dt}")
        else:
            # Time derivative
            time_str = "\\left. \\frac{\\partial}{\\partial \\xi_{1}} \\eta_{" + str(tangent_index) + "}{\\left(\\xi_{1},u,v \\right)} \\right|_{\\substack{ \\xi_{1}=t }}"
            output_str = output_str.replace(time_str,"\\frac{\\partial\\eta_"+ str(tangent_index)+"}{\\partial t}")            
            state_str = ""
            # Loop through the states and find the partial derivatives
            for state_index in range(len(states)):
                # Extract the string for the state at hand
                state_str = latex(states[state_index])
                state_str = state_str.replace("{\\left(t \\right)}","")
                # Replace the partial derivative at hand
                output_str = output_str.replace("\\frac{d}{d " + state_str +"} \\eta_{"+ str(tangent_index) +"}","\\frac{\\partial\\eta_{"+ str(tangent_index) +"}}{\\partial "+ state_str +"}")            
    # Return the output string
    return tangent_str,output_str

# Test output from automated function

In [5]:
tangent_list,lin_sym_original = lin_sym_conds_fibre_preserving(t,states,omega_list)    
tangent_str,output_str = latex_printing_of_lin_sym(t,states,tangent_list,lin_sym_original)
print("Printing the output")
print(tangent_str)
print(output_str)

Printing the output


The tangents are:
$$\left[ \xi{\left(t \right)}, \  \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}, \  \eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}\right]$$


The linearised symmetry conditions are:

\begin{align*}
a \left(u - 1\right) v \frac{\partial\eta_{1}}{\partial v} - \left(1 - v\right) \eta_{1} + \left(1 - v\right) u \frac{\partial\eta_{1}}{\partial u} - \left(1 - v\right) u \frac{d\xi}{dt} + \eta_{2} u + \frac{\partial\eta_1}{\partial t}&=0,\\
- a \left(u - 1\right) \eta_{2} + a \left(u - 1\right) v \frac{\partial\eta_{2}}{\partial v} - a \left(u - 1\right) v \frac{d\xi}{dt} - a \eta_{1} v + \left(1 - v\right) u \frac{\partial\eta_{2}}{\partial u} + \frac{\partial\eta_2}{\partial t}&=0,\\
\end{align*}




Printing the output


The tangents are:
$$\left[ \xi{\left(t \right)}, \  \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}, \  \eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}\right]$$


The linearised symmetry conditions are:

\begin{align*}
a \left(u - 1\right) v \frac{\partial\eta_{1}}{\partial v} - \left(1 - v\right) \eta_{1} + \left(1 - v\right) u \frac{\partial\eta_{1}}{\partial u} - \left(1 - v\right) u \frac{d\xi}{dt} + \eta_{2} u + \frac{\partial\eta_1}{\partial t}&=0,\\
- a \left(u - 1\right) \eta_{2} + a \left(u - 1\right) v \frac{\partial\eta_{2}}{\partial v} - a \left(u - 1\right) v \frac{d\xi}{dt} - a \eta_{1} v + \left(1 - v\right) u \frac{\partial\eta_{2}}{\partial u} + \frac{\partial\eta_2}{\partial t}&=0.\\
\end{align*}

Ok, so the next step is to try to formulate the *determining equations*. This is done in two steps:

1. Solve the first equation above for $\eta_2$ and then plug in this equation into the second equation.
2. Organise the resulting second order PDE in terms of the derivative (as the derivatives are linearly independent).

The resulting equations are referred to as the *determining equations*. Let's see if we can manipulate these, and derive the determining equations.



In [29]:
# Solve equations for the second component
# Extract equation
lin_sym_temp_1 = lin_sym_original[0]
print("Equation for linearised symmetry equation\n")
print(lin_sym_temp_1)
# Solve this equation for eta_2
eta_2_solution = [tangent_list[2],solve(lin_sym_temp_1,tangent_list[1])[0]]
print("\n\nSolve this equation for the second tangent\n")
print(eta_2_solution)
# Substitute this into the second linearised symmetry condition and simplify
lin_sym_temp_2 = expand(lin_sym_original[1].subs(eta_2_solution[0],eta_2_solution[1]).doit())
print("\n\nIf we substitute this solution into the second equation and simplify this is what we get:\n")
print("$$" + latex(lin_sym_temp_2) + "=0$$")

Equation for linearised symmetry equation

a*(u(t) - 1)*v(t)*Derivative(eta_1(t, u(t), v(t)), v(t)) - (1 - v(t))*eta_1(t, u(t), v(t)) + (1 - v(t))*u(t)*Derivative(eta_1(t, u(t), v(t)), u(t)) - (1 - v(t))*u(t)*Derivative(xi(t), t) + eta_2(t, u(t), v(t))*u(t) + Subs(Derivative(eta_1(_xi_1, u(t), v(t)), _xi_1), _xi_1, t)


Solve this equation for the second tangent

[eta_2(t, u(t), v(t)), (-a*u(t)*v(t)*Derivative(eta_1(t, u(t), v(t)), v(t)) + a*v(t)*Derivative(eta_1(t, u(t), v(t)), v(t)) - eta_2(t, u(t), v(t))*u(t) + u(t)*v(t)*Derivative(eta_1(t, u(t), v(t)), u(t)) - u(t)*v(t)*Derivative(xi(t), t) - u(t)*Derivative(eta_1(t, u(t), v(t)), u(t)) + u(t)*Derivative(xi(t), t) - Subs(Derivative(eta_1(_xi_1, u(t), v(t)), _xi_1), _xi_1, t))/(v(t) - 1)]


If we substitute this solution into the second equation and simplify this is what we get:

$$\frac{a^{2} u^{2}{\left(t \right)} v^{2}{\left(t \right)} \frac{d}{d v{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{

So this is what our equation looks like!

$$\frac{a^{2} u^{2}{\left(t \right)} v^{2}{\left(t \right)} \frac{d}{d v{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} - \frac{2 a^{2} u{\left(t \right)} v^{2}{\left(t \right)} \frac{d}{d v{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} + \frac{a^{2} v^{2}{\left(t \right)} \frac{d}{d v{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} - \frac{a^{2} u^{2}{\left(t \right)} v^{2}{\left(t \right)} \frac{d^{2}}{d v{\left(t \right)}^{2}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} + \frac{2 a^{2} u{\left(t \right)} v^{2}{\left(t \right)} \frac{d^{2}}{d v{\left(t \right)}^{2}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - \frac{a^{2} v^{2}{\left(t \right)} \frac{d^{2}}{d v{\left(t \right)}^{2}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - a \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)} v{\left(t \right)} - a u{\left(t \right)} v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)} + a v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)} + \frac{a \eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)} u^{2}{\left(t \right)} v{\left(t \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} - \frac{a \eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)} u{\left(t \right)} v{\left(t \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} - \frac{a u^{2}{\left(t \right)} v^{2}{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} + \frac{a u^{2}{\left(t \right)} v^{2}{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} + \frac{a u^{2}{\left(t \right)} v{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} - \frac{a u^{2}{\left(t \right)} v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} + \frac{a u{\left(t \right)} v^{2}{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} - \frac{a u{\left(t \right)} v^{2}{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} - \frac{a u{\left(t \right)} v{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} + \frac{a u{\left(t \right)} v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} + \frac{a u{\left(t \right)} v{\left(t \right)} \left. \frac{\partial}{\partial \xi_{1}} \eta_{1}{\left(\xi_{1},u{\left(t \right)},v{\left(t \right)} \right)} \right|_{\substack{ \xi_{1}=t }}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} - \frac{a v{\left(t \right)} \left. \frac{\partial}{\partial \xi_{1}} \eta_{1}{\left(\xi_{1},u{\left(t \right)},v{\left(t \right)} \right)} \right|_{\substack{ \xi_{1}=t }}}{v^{2}{\left(t \right)} - 2 v{\left(t \right)} + 1} + \frac{a \eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)} u^{2}{\left(t \right)}}{v{\left(t \right)} - 1} - \frac{a \eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)} u{\left(t \right)}}{v{\left(t \right)} - 1} + \frac{2 a u^{2}{\left(t \right)} v^{2}{\left(t \right)} \frac{d^{2}}{d v{\left(t \right)}d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - \frac{a u^{2}{\left(t \right)} v{\left(t \right)} \frac{d}{d v{\left(t \right)}} \eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - \frac{2 a u^{2}{\left(t \right)} v{\left(t \right)} \frac{d^{2}}{d v{\left(t \right)}d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} + \frac{a u^{2}{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - \frac{a u^{2}{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}}{v{\left(t \right)} - 1} + \frac{a u{\left(t \right)} v^{2}{\left(t \right)} \frac{d}{d v{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - \frac{2 a u{\left(t \right)} v^{2}{\left(t \right)} \frac{d^{2}}{d v{\left(t \right)}d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - \frac{a u{\left(t \right)} v{\left(t \right)} \frac{d}{d v{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} + \frac{a u{\left(t \right)} v{\left(t \right)} \frac{d}{d v{\left(t \right)}} \eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} + \frac{2 a u{\left(t \right)} v{\left(t \right)} \frac{d^{2}}{d v{\left(t \right)}d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - \frac{a u{\left(t \right)} v{\left(t \right)} \left. \frac{\partial^{2}}{\partial v{\left(t \right)}\partial \xi_{1}} \eta_{1}{\left(\xi_{1},u{\left(t \right)},v{\left(t \right)} \right)} \right|_{\substack{ \xi_{1}=t }}}{v{\left(t \right)} - 1} - \frac{a u{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} + \frac{a u{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}}{v{\left(t \right)} - 1} + \frac{a u{\left(t \right)} \left. \frac{\partial}{\partial \xi_{1}} \eta_{1}{\left(\xi_{1},u{\left(t \right)},v{\left(t \right)} \right)} \right|_{\substack{ \xi_{1}=t }}}{v{\left(t \right)} - 1} + \frac{a v{\left(t \right)} \left. \frac{\partial^{2}}{\partial v{\left(t \right)}\partial \xi_{1}} \eta_{1}{\left(\xi_{1},u{\left(t \right)},v{\left(t \right)} \right)} \right|_{\substack{ \xi_{1}=t }}}{v{\left(t \right)} - 1} - \frac{a \left. \frac{\partial}{\partial \xi_{1}} \eta_{1}{\left(\xi_{1},u{\left(t \right)},v{\left(t \right)} \right)} \right|_{\substack{ \xi_{1}=t }}}{v{\left(t \right)} - 1} + \left. \frac{\partial}{\partial \xi_{1}} \eta_{2}{\left(\xi_{1},u{\left(t \right)},v{\left(t \right)} \right)} \right|_{\substack{ \xi_{1}=t }} + \frac{\eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)} u{\left(t \right)} v{\left(t \right)}}{v{\left(t \right)} - 1} - \frac{\eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)} u{\left(t \right)}}{v{\left(t \right)} - 1} - \frac{u^{2}{\left(t \right)} v^{2}{\left(t \right)} \frac{d^{2}}{d u{\left(t \right)}^{2}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} + \frac{2 u^{2}{\left(t \right)} v{\left(t \right)} \frac{d^{2}}{d u{\left(t \right)}^{2}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} + \frac{u^{2}{\left(t \right)} v{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - \frac{u^{2}{\left(t \right)} \frac{d^{2}}{d u{\left(t \right)}^{2}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - \frac{u^{2}{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{2}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - \frac{u{\left(t \right)} v^{2}{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} + \frac{u{\left(t \right)} v^{2}{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}}{v{\left(t \right)} - 1} + \frac{2 u{\left(t \right)} v{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} - \frac{2 u{\left(t \right)} v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}}{v{\left(t \right)} - 1} + \frac{u{\left(t \right)} v{\left(t \right)} \left. \frac{\partial^{2}}{\partial u{\left(t \right)}\partial \xi_{1}} \eta_{1}{\left(\xi_{1},u{\left(t \right)},v{\left(t \right)} \right)} \right|_{\substack{ \xi_{1}=t }}}{v{\left(t \right)} - 1} - \frac{u{\left(t \right)} \frac{d}{d u{\left(t \right)}} \eta_{1}{\left(t,u{\left(t \right)},v{\left(t \right)} \right)}}{v{\left(t \right)} - 1} + \frac{u{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}}{v{\left(t \right)} - 1} - \frac{u{\left(t \right)} \left. \frac{\partial^{2}}{\partial u{\left(t \right)}\partial \xi_{1}} \eta_{1}{\left(\xi_{1},u{\left(t \right)},v{\left(t \right)} \right)} \right|_{\substack{ \xi_{1}=t }}}{v{\left(t \right)} - 1}=0$$
And now we want to identify derivatives with respect to $\eta_1$ and $\eta_2$ how do we go about achieving this?

# Attempt 1
Perhaps the easiest way is in fact to convert the symbolic object to a string? We can then separate all terms with respect to the plus signs? Our maybe it is better to split them up before as a symbolic object using args. Anyhow, We should then be able to find all string with the keyword "Derivative". And then we should be able to sort out all types of derivatives we have and lastly save all corresponding terms as well. 

In [31]:
# Extract one term for example
print(lin_sym_temp_2.args[0])
# Extract another
print(lin_sym_temp_2.args[1])

a*v(t)*Derivative(xi(t), t)
u(t)*Derivative(xi(t), t)/(v(t) - 1)
